In [ ]:
# -*- coding: utf-8 -*-
"""Falcon_Hugging_face_adapter.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/15cOnEWMSFnfpXdAg5Sc9yaLTMB7Nu_ID
"""

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [ ]:
!pip install -q einops

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

#dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] == 8 else torch.float16

model_id = "falcon-7B-instruct-600steps"
#model_id = "falcon-40B-instruct-600steps"
config = PeftConfig.from_pretrained(model_id)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        return_dict=True,
        device_map = "auto",
        #use_ram_optimized_load=False,
        offload_folder="./tmp",
        #torch__dtype = dtype,
        #offload_dir = '.',
        #load_in_8bit = True,
        #device_map = 'auto',
        trust_remote_code = True)

In [ ]:
model = PeftModel.from_pretrained(model, model_id, offload_dir="./tmp")

In [ ]:
model

In [ ]:
m = model.merge_and_unload()

In [ ]:
m

In [ ]:
#m.push_to_hub('falcon-7B-instruct-300steps-merged', use_auth_token=True, create_pr=1)
m.push_to_hub('falcon-7B-instruct-600steps-merged', use_auth_token=True, create_pr=1)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
#tokenizer.push_to_hub('/falcon-7B-instruct-300steps-merged', use_auth_token=True, create_pr=1)
tokenizer.push_to_hub('/falcon-7B-instruct-600steps-merged', use_auth_token=True, create_pr=1)